# Tier Wrapper generator

The following notebook generates the wrapping code to make C++ available to the pyclesperanto python code.
This notebook is to be runned for each Tiers (1,2,3, etc.) and will generate the corresponding wrapping code.
It only need to be runned when a Tier is updated.

__*WARNING*__: this can break the code if the python package is not update consequently. Please check the code and run the tests before pushing.

In [17]:
import glob
import os
import sys
import re
import requests
import numpy as np

# define __file__ if it is not defined
if '__file__' not in globals():
    __file__ = os.path.abspath('generate-tiers-package.ipynb')

# get platform linux, windows, darwin
platform = sys.platform
# get platform architecture x86_64, x86, armv7l
architecture = os.uname().machine
# get the python version
python_version = sys.version_info

Get the list of hpp files in the tiers directory and sort them by name.

In [18]:
# get first argument of the script
tier = 1
file = 'clic/include/tier' + str(tier) + '.hpp'
repo = 'https://github.com/clEsperanto/CLIc_prototype.git'
branch = 'add-cuda-backend'

# # get current directory
# current_dir = os.path.dirname(os.path.abspath(__file__))
# # get the relative path to directory and add 'tier' plus the number of the tier
# tier_dir = os.path.join(current_dir, '..', '_skbuild', 'linux-x86_64-3.10', 'cmake-build', '_deps', 'clic_lib-src', 'clic', 'include', 'tier' + str(tier))
# # list all files in the tier_dir
# files = glob.glob(tier_dir + '.hpp')
# print(files)

Define a set of function to parse the hpp files and extract the information we need.

In [19]:
def read_file(file):
    with open(file, 'r') as f:
        content = f.read()
    return content

def read_from_repo(file_path, repo_url='https://github.com/clEsperanto/CLIc_prototype.git', branch ='master'):
    # Construct the raw file URL
    repo_url = repo_url.split('.git')[0].split('github.com/')[1]
    raw_file_url = f"https://raw.githubusercontent.com/{repo_url}/{branch}/{file_path}"
    
    # Make an HTTP GET request to the raw file URL
    response = requests.get(raw_file_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Return the content of the file
        return response.text
    

def parse_file(file_content):
    # Regular expression pattern to extract function names, parameter names, and parameter types
    pattern = r"(\w+_func)\s*\(([^)]*)\)\s*->\s*(\w+::\w+|\w+)"

    # Extract function names, parameter names, and parameter types using regular expression
    matches = re.findall(pattern, file_content, re.MULTILINE)

    # Create a dictionary to store function names and their corresponding parameter names and types
    function_parameters = {}

    # Process each match to extract function name, parameter names, and parameter types
    for match in matches:
        function_name, parameters, return_type = match
        parameter_names = []
        parameter_types = []
        for param in parameters.split(","):
            param = param.strip()
            if param.startswith("const"):
                param = param[5:].strip()
            param_parts = re.findall(r"(\w+::\w+|\w+)", param)
            if len(param_parts) == 2:
                parameter_types.append(param_parts[0])
                parameter_names.append(param_parts[1])
            else:
                parameter_types.append(None)
                parameter_names.append(None)
        function_parameters[function_name] = {
            'parameter_names': parameter_names,
            'parameter_types': parameter_types,
            'return_type': return_type
        }
    
    return function_parameters


def generate_wrapper_function(tier, function_name, parameters_name):
    code_template = """
m.def(\"_{func_name}\", &cle::tier{tier}::{func_name}_func, "Call {func_name} from C++.",
    py::return_value_policy::take_ownership,
    {param_bindings});
"""
    function_name = function_name.replace('_func', '')
    parameters_binding = ", ".join([f"py::arg(\"{param_name}\")" for param_name in parameters_name])
    return code_template.format(func_name=function_name, tier=tier, param_bindings=parameters_binding)


def generate_python_function(tier, function_name, parameters_name, parameters_type, return_type):
    code_template = """
@plugin_function
def {func_name}(
    {param_defines}
) -> {return_type}:
    from ._pyclesperanto import _{func_name} as op

    return op(
        {param_bindings}
    )
"""
    return_type = return_type.replace('::Pointer', '').replace('Array', 'Image')
    function_name = function_name.replace('_func', '')
    parameter_binding_list = []
    parameter_defines_list = []

    for param_name, param_type in zip(parameters_name, parameters_type):
        name = param_name.replace('src', 'input_image').replace('dst', 'output_image')
        if param_type in ['float', 'int']:
            parameter_binding_list.append(f"{param_name}={param_type}({name})")
        else:
            parameter_binding_list.append(f"{param_name}={name}")

    parameters_name = np.roll(parameters_name, -1)
    parameters_type = np.roll(parameters_type, -1)
    for param_name, param_type in zip(parameters_name, parameters_type):
        name = param_name.replace('src', 'input_image').replace('dst', 'output_image')
        type_declare = param_type.replace('::Pointer', '').replace('Array', 'Image')
        default_value = ' = None'
        if param_name.find('src') != -1:
            default_value = ''
        if param_type in ['float', 'int']:
            default_value = ' = 1'
        parameter_defines_list.append(f'{name}: {type_declare}{default_value}')

    parameter_defines = ",\n\t".join(
        parameter_defines_list
    )
    parameter_bindings = ",\n\t\t".join(
        parameter_binding_list
    )

    return code_template.format(func_name=function_name, param_defines=parameter_defines, param_bindings=parameter_bindings, return_type=return_type)


def generate_wrapper_code(file, tier, wrapper_functions):
    code_template = """
// this code is auto-generated by the script 'pyclesperanto_autogen_tier_script.ipynb'.
// Do not edit manually. Instead, edit the script and run it again.
    
#include "pycle_wrapper.hpp"
#include "tier{tier}.hpp"

namespace py = pybind11;

auto tier{tier}_(py::module &m) -> void {{

    {wrapper_functions}

}}
"""
    code = code_template.format(tier=tier, wrapper_functions="\n    ".join(wrapper_functions))
    with open(file, 'w') as f:
        f.write(code)

def generate_python_code(file, tier, python_functions):
    code_template = """
# this code is auto-generated by the script 'pyclesperanto_autogen_tier_script.ipynb'.
# Do not edit manually. Instead, edit the script and run it again.

from ._core import Device
from ._array import Image, Array
from ._decorators import plugin_function

{python_functions}
"""
    code = code_template.format(python_functions="\n".join(python_functions))
    with open(file, 'w') as f:
        f.write(code)

For each files, we extract the name of the function it contains as well as its signature. From both we build the wrapper code to make the function visible in Python.

In [20]:
wrapper_functions = []

# file_content = read_file(files[0])
file_content = read_from_repo(file, repo, branch)
function_parameters = parse_file(file_content)

wrapper_functions = []
python_functions = []
for function_name, parameters in function_parameters.items():
    wrapper_functions.append(generate_wrapper_function(tier, function_name, parameters['parameter_names']))
    python_functions.append(generate_python_function(tier, function_name, parameters['parameter_names'], parameters['parameter_types'], parameters['return_type']))


https://raw.githubusercontent.com/clEsperanto/CLIc_prototype/add-cuda-backend/clic/include/tier1.hpp
#ifndef __INCLUDE_TIER1_HPP
#define __INCLUDE_TIER1_HPP

#include "tier0.hpp"

namespace cle::tier1
{

auto
absolute_func(const Device::Pointer & device, const Array::Pointer & src, Array::Pointer dst) -> Array::Pointer;

auto
add_images_weighted_func(const Device::Pointer & device,
                         const Array::Pointer &  src0,
                         const Array::Pointer &  src1,
                         Array::Pointer          dst,
                         const float &           factor0,
                         const float &           factor1) -> Array::Pointer;

auto
add_image_and_scalar_func(const Device::Pointer & device,
                          const Array::Pointer &  src,
                          Array::Pointer          dst,
                          const float &           scalar) -> Array::Pointer;

// average_distance_of_n_far_off_distances_func
// average_dista

We create the wrapper file to contains the Tier functions currently processed. And for each function, we add the wrapper code to the file.

In [ ]:
# get the path to the current directory
current_dir = os.path.dirname(os.path.abspath(__file__))
cpp_file = os.path.join(current_dir, '..', 'wrapper', 'tier' + str(tier) + '_.cpp')
py_file = os.path.join(current_dir, '..', 'pyclesperanto', '_tier' + str(tier) + '.py')

# if the file already exists, rename it to cleTierX_old.cpp
if os.path.exists(cpp_file):
    os.rename(cpp_file, cpp_file.replace('.cpp', '.old_cpp'))
if os.path.exists(py_file):
    os.rename(py_file, py_file.replace('.py', '.old_py'))    

generate_wrapper_code(cpp_file, tier, wrapper_functions)
generate_python_code(py_file, tier, python_functions)